In [1]:
import numpy as np
import pandas as pd
import pydicom
%matplotlib inline
import matplotlib.pyplot as plt
import keras 
from keras.models import model_from_json
from skimage.transform import resize
from keras.utils.vis_utils import plot_model

Using TensorFlow backend.


In [2]:
# This function reads in a .dcm file, checks the important fields for our device, and returns a numpy array
# of just the imaging data
def check_dicom(filename): 
    
    print('Load file {} ...'.format(filename))
    ds = pydicom.dcmread(filename)
    modality = ds.Modality
    bodyPart = ds.BodyPartExamined
    position = ds.PatientPosition
    if ((modality == 'DX') & (bodyPart == 'CHEST') & (position in ['PA', 'AP'])):
        return ds.pixel_array
    
    print("Invalid file: modality is not 'DX', or body part is not 'CHEST', or position is not 'AP' or 'PA'.\n")
    return None
    
    
# This function takes the numpy array output by check_dicom and 
# runs the appropriate pre-processing needed for our model input
def preprocess_image(img, img_mean, img_std, img_size): 
    
    #img = (img - img_mean) / (img_std * 255)
    img = img / 255
    img = resize(img, (img_size[1], img_size[2]), anti_aliasing=True)
    
    proc_img = np.zeros(img_size)
    proc_img[0,:,:,0] = img
    proc_img[0,:,:,1] = img
    proc_img[0,:,:,2] = img
    
    return proc_img

# This function loads in our trained model w/ weights and compiles it 
def load_model(model_path, weight_path):
    # open and load model
    with open(model_path) as json_mod:
        model = model_from_json(json_mod.read())
    # load weights
    model.load_weights(weight_path)
    plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)
    
    return model

# This function uses our device's threshold parameters to predict whether or not
# the image shows the presence of pneumonia using our trained model
def predict_image(model, img, thresh): 
    
    prediction = int(model.predict(img) > thresh)
    
    return prediction 

In [3]:
test_dicoms = ['test1.dcm','test2.dcm','test3.dcm','test4.dcm','test5.dcm','test6.dcm']

model_path = "./model.json" #path to saved model
weight_path = "./xray_class_my_model.best.hdf5" #path to saved best weights

IMG_SIZE=(1,224,224,3) # This might be different if you did not use vgg16
img_mean = 0 # loads the mean image value they used during training preprocessing
img_std = 1 # loads the std dev image value they used during training preprocessing

my_model = load_model(model_path, weight_path) #loads model
thresh = 0.34 #loads the threshold they chose for model classification 

# use the .dcm files to test your prediction
for i in test_dicoms:
    
    img = np.array([])
    img = check_dicom(i)
    
    if img is None:
        continue
        
    img_proc = preprocess_image(img, img_mean, img_std, IMG_SIZE)
    pred = predict_image(my_model, img_proc, thresh)
    print(pred, "\n")

Load file test1.dcm ...
0 

Load file test2.dcm ...
1 

Load file test3.dcm ...
1 

Load file test4.dcm ...
Invalid file: modality is not 'DX', or body part is not 'CHEST', or position is not 'AP' or 'PA'.

Load file test5.dcm ...
Invalid file: modality is not 'DX', or body part is not 'CHEST', or position is not 'AP' or 'PA'.

Load file test6.dcm ...
Invalid file: modality is not 'DX', or body part is not 'CHEST', or position is not 'AP' or 'PA'.

